In [78]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import hvplot.pandas
from pathlib import Path
from numpy import mean
from numpy import std
from numpy import cov
from scipy.stats import pearsonr
import requests
import urllib.request
import json
import pprint
from dotenv import load_dotenv

%matplotlib inline

In [79]:
load_dotenv()
access_token = os.getenv("CQ_API_KEY")
headers = { 'Authorization': 'Bearer ' + access_token }

print(type(headers))

<class 'dict'>


In [80]:
URL = "https://api.cryptoquant.com/v1/btc/exchange-flows/netflow?exchange=all_exchange&window=day&from=20150101&limit=100"
response = requests.get(URL, headers=headers)

netflow_data = response.json()
netflow_df = pd.DataFrame(data=netflow_data['result']['data'])
netflow_df['date'] = pd.to_datetime(netflow_df['date'])
netflow_df['netflow_total'] = netflow_df['netflow_total'].astype(str).astype('float64')
netflow_df = netflow_df.dropna()

netflow_df.describe






<bound method NDFrame.describe of          date  netflow_total
0  2021-01-21    -153.516795
1  2021-01-20    3643.276701
2  2021-01-19  -14814.094964
3  2021-01-18    6798.665563
4  2021-01-17   -5060.862951
5  2021-01-16    -355.418920
6  2021-01-15   17624.149371
7  2021-01-14   -3318.766133
8  2021-01-13    4514.323077
9  2021-01-12      83.452431
10 2021-01-11    5264.572792
11 2021-01-10    9508.089746
12 2021-01-09   -2953.703428
13 2021-01-08    7752.349472
14 2021-01-07   -1344.990247
15 2021-01-06   -2528.676266
16 2021-01-05   -1405.490292
17 2021-01-04    1915.942121
18 2021-01-03   17355.683806
19 2021-01-02  -32492.928409
20 2021-01-01   -3298.734090
21 2020-12-31   -8215.578255
22 2020-12-30    2405.820970
23 2020-12-29    -166.657571
24 2020-12-28    1801.899760
25 2020-12-27   11508.658954
26 2020-12-26    3503.867941
27 2020-12-25    9390.618269
28 2020-12-24   10290.831612
29 2020-12-23   -5087.449641
..        ...            ...
70 2020-11-12   -7708.799008
71 2020-1

In [81]:
URL2 = "https://api.cryptoquant.com/v1/btc/market-data/price-usd?window=day&from=20150101&limit=100"
response2 = requests.get(URL2, headers=headers)

price_data = response2.json()

price_df = pd.DataFrame(data=price_data['result']['data'])
price_df = price_df.drop(['price_usd_high','price_usd_low','price_usd_open'], axis = 1)
price_df['date'] = pd.to_datetime(price_df['date'])
price_df['price_usd_close'] = price_df['price_usd_close'].astype(str).astype('float64')
price_df = price_df.dropna()

price_df.describe


<bound method NDFrame.describe of          date  price_usd_close
0  2021-01-21     30848.486667
1  2021-01-20     35481.915000
2  2021-01-19     35888.901667
3  2021-01-18     36641.193333
4  2021-01-17     35830.791667
5  2021-01-16     35991.390000
6  2021-01-15     36761.818333
7  2021-01-14     39151.978333
8  2021-01-13     37383.333333
9  2021-01-12     34036.688333
10 2021-01-11     35437.078333
11 2021-01-10     38175.311667
12 2021-01-09     40149.151667
13 2021-01-08     40616.906667
14 2021-01-07     39470.805000
15 2021-01-06     36815.308333
16 2021-01-05     33994.421667
17 2021-01-04     32017.403333
18 2021-01-03     33034.825000
19 2021-01-02     32195.855000
20 2021-01-01     29355.463333
21 2020-12-31     28937.490000
22 2020-12-30     28882.456000
23 2020-12-29     27383.052000
24 2020-12-28     27061.882000
25 2020-12-27     26269.252000
26 2020-12-26     26481.480000
27 2020-12-25     24709.222000
28 2020-12-24     23738.966000
29 2020-12-23     23232.500000
..   

In [82]:
covariance = cov(netflow_df['netflow_total'],price_df['price_usd_close'])
corr, _ = pearsonr(netflow_df['netflow_total'], price_df['price_usd_close'])
print('Pearsons correlation: %.3f' % corr)
#print(type(netflow_df['netflow_total']))

#price_to_netflow_df = pd.concat([price_df,netflow_df['netflow_total']],ignore_index=False, axis=1, sort=False)
#price_to_netflow_df = price_to_netflow_df.drop_duplicates()

#price_to_netflow_df.head(10)

Pearsons correlation: 0.213


In [83]:
#price_to_netflow_df.plot(x="date", y=["price_usd_close", "netflow_total"])
#price_to_netflow_df.set_index('date').plot(figsize=(10,5), grid=True)

In [84]:
#fig, ax1 = plt.subplots()

#color = 'tab:red'
#ax1.set_xlabel('Date')
#ax1.set_ylabel('BTC Netflow', color=color)
#ax1.plot(price_to_netflow_df['date'], price_to_netflow_df['netflow_total'], color=color)
#ax1.tick_params(axis='y', labelcolor=color)

#ax2 = ax1.twinx()  

#color = 'tab:blue'
#ax2.set_ylabel('Price USD', color=color)  
#ax2.plot(price_to_netflow_df['date'], price_to_netflow_df['price_usd_close'], color=color)
#ax2.tick_params(axis='y', labelcolor=color)

#fig.tight_layout()  # otherwise the right y-label is slightly clipped

#plt.show()

In [85]:
def pearsons_coefficient(series1,series2):
    
  #Returns pearsons coefficient (int) between 2 dataframes columns.

       # Parameters: 
            ## series1 (object): First series to compare
            ## series2 (object): Second series to compare
            ## df1_column (object): Series from df1 we wish to compare
            ## df2_column (object): Series from df2 we wish to compare

    

    covariance = cov(series1,series2)
    corr, _ = pearsonr(series1,series2)
    print('Pearsons correlation: %.3f' % corr)
    return corr



In [86]:
test_df = pearsons_coefficient(netflow_df['netflow_total'],price_df['price_usd_close'])

Pearsons correlation: 0.213


In [87]:
def get_onchain_data(access_token,ticker,fromdate,limit):
    '''
    Returns on-chain data (Netflow)n in new df
        Parameters:
            access_token (string): API key
            ticker (string) = Ticker for Desired Coin
            fromdate (string) = Starting date of period desired
            limit (int) = number of data points     
    '''
    #Initialize dotenv function, pull API key, build URL
    headers = {'Authorization': 'Bearer ' + access_token}
    URL = f'https://api.cryptoquant.com/v1/{ticker}/exchange-flows/netflow?exchange=all_exchange&window=day&from={fromdate}&limit={limit}'

    # Initialize Dictionary
    response = requests.get(URL, headers=headers)
    
    netflow_data = response.json()

    #Transform to Dataframe and adjust data type
    netflow_df = pd.DataFrame(data=netflow_data['result']['data'])
    netflow_df['date'] = pd.to_datetime(netflow_df['date'])
    netflow_df['netflow_total'] = netflow_df['netflow_total'].astype(str).astype('float64')
    return netflow_df


In [89]:
#test1_df = get_onchain_data(access_token,"btc","20190101","100")

#print(test1_df)